# <mark>*Librairies Importation* </mark>

In [2]:
!nvidia-smi

Sun Mar 26 21:15:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## <mark> *set up the environment* </mark>

In [3]:
#basic functionnality
import matplotlib.pyplot as plt
import numpy as np
#image processing functionnality
# import cv2 as cv
# import PIL
# import pathlib 
# import os
# import random
# import glob
#tensorflow functionnality
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten,Rescaling
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# <mark>*Build the model* </mark>

In [4]:
#os.chdir('/content/drive/MyDrive/Colab Notebooks/projects/melanoma_datasets')
dataset_url="/content/drive/MyDrive/Colab Notebooks/projects/melanoma_datasets"

test_path="/content/drive/MyDrive/project/final-project/datasets/melanoma_cancer_dataset/test"
train_path="/content/drive/MyDrive/project/final-project/datasets/melanoma_cancer_dataset/train"
valid_path="/content/drive/MyDrive/project/final-project/datasets/melanoma_cancer_dataset/validation"

In [10]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path,target_size=(300,300),classes=['malignant','benign'],batch_size=30)
test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path,target_size=(300,300),classes=['malignant','benign'],batch_size=30)
valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=valid_path,target_size=(300,300),classes=['malignant','benign'],batch_size=30)

Found 13891 images belonging to 2 classes.
Found 1010 images belonging to 2 classes.
Found 3000 images belonging to 2 classes.


In [11]:
#inceptionV3 is also known as googLeNet
scale=1./255
inceptionV3_model = Sequential()
pretrained_model = InceptionV3(
    include_top=False,
    input_shape=(300,300,3),  #googleNet model take 299x299 as resolution that why we have reshape the images
    pooling = 'max',
    weights='imagenet'
)
#i'll be using transfert learning so i'm gonna freeze the layers
for layer in pretrained_model.layers:
  layer.trainable=False
# add layers
inceptionV3_model.add(pretrained_model)
inceptionV3_model.add(layers.Rescaling(scale))
inceptionV3_model.add(Flatten())
inceptionV3_model.add(Dense(1024,activation='relu'))
inceptionV3_model.add(Dense(512,activation='relu'))
inceptionV3_model.add(Dense(256,activation='relu'))
inceptionV3_model.add(Dense(2,activation='sigmoid'))
inceptionV3_model.summary()
tb_call_back=tf.keras.callbacks.TensorBoard(log_dir="/content/drive/MyDrive/project/final-project/callbacks-logs/InceptionV3",histogram_freq=1)
inceptionV3_model.compile(
      optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 rescaling_1 (Rescaling)     (None, 2048)              0         
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_5 (Dense)             (None, 512)               524800    
                                                                 
 dense_6 (Dense)             (None, 256)               131328    
                                                                 
 dense_7 (Dense)             (None, 2)                

# <mark>*train the model* </mark>

In [1]:
history = inceptionV3_model.fit(
    train_batches,
    validation_data=valid_batches,
    epochs=100,
    verbose=1,
    callbacks=[tb_call_back]
)

NameError: ignored

# <mark>*Plot training (learning state) graph* </mark>

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train','validation'])
plt.show()

NameError: ignored

# <mark>*TensorBoard to see training evolution* </mark>

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="/content/drive/MyDrive/project/final-project/callbacks-logs/InceptionV3"